## 1. Web Access

### 1. Modules & Variables

In [1]:
! pip install undetected_chromedriver

# 원격 조종 시에도 구글 로그인 가능
# 참고 링크
# https://github.com/ultrafunkamsterdam/undetected-chromedriver


In [7]:
import time
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC  # noqa

import undetected_chromedriver as uc

from bs4 import BeautifulSoup


In [8]:
dict_gAccounts = {
    'kjaehw0207': 'flapaqj1!%',
    'woghks.study': 'ans_entrance00'
}

account_1 = list(dict_gAccounts.keys())[0]
password_1 = dict_gAccounts[account_1]

account_2 = list(dict_gAccounts.keys())[1]
password_2 = dict_gAccounts[account_2]


### 2. Web Access

In [9]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')


In [10]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
btn_sign_in = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')))
btn_sign_in.click()

# 계정 이메일 입력
input_account = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
input_account.send_keys(account_2)

# [다음] 버튼 클릭해 패스워드 입력
btn_next_password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
btn_next_password.click()

# 계정 비밀번호 입력
input_password = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
input_password.send_keys(password_2)

# [다음] 버튼 입력해 로그인 클릭
btn_next_sign_in = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
btn_next_sign_in.click()


In [11]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지
try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


맞춤 페이지 없음. 시청기록 수집을 시작합니다.


### 3. Data Collection

#### 1) Functions & Variables

In [12]:
def get_day(weekday=datetime.today().weekday()):
    list = ['월', '화', '수', '목', '금', '토', '일']
    return list[weekday]


def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d').strftime('%Y-%m-%d')


In [13]:
# 항목별 Class 이름


# 시청일자 - 목록 content에서 [data-date] 값 추출
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>

nm_class_adfilter = "SiEggd"  # 채널표기 유무: 없는 경우 광고 / tag = <div>

day_today = get_day()
print(day_today)


일


#### 2) DataFrame

In [14]:
import pandas as pd
columns_rec = ['title', 'category', 'running_time',
               'watch_date', 'watch_time',  'watched_YN']
df_record = pd.DataFrame(columns=columns_rec)
df_record.head()


,title,category,running_time,watch_date,watch_time,watched_YN


In [15]:
columns_ser = ['searchword', 'search_date', 'search_time']
df_search = pd.DataFrame(columns=columns_ser)
df_search.head()


,searchword,search_date,search_time


#### 3) Web Data Collecting

In [16]:
# 목표 갯수만큼 시청 목록(레코드) 호출

num_target_data = 800 #int(input("수집할 레코드 수를 100개 단위로 입력해주세요."))
num_press_end = int((num_target_data / 100)-1)

for n in range(num_press_end):
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
contents = soup.find_all('c-wiz', class_='xDtZAf')
print(f'호출된 레코드 수: {len(contents)}')


호출된 레코드 수: 800


In [17]:
# 레코드 분류 및 카테고리/시청여부 입력

idx_rec = 0 # df_record의 idx
idx_ser = 0 # df_search의 idx
num_ad = 0 # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'): # 채널명이 있는 경우: 
        if content.find('div', class_=nm_class_RunningTime):
            print(f"{idx}: Watch Record")

            title = content.find('a', class_=nm_class_title).text
            category = input(f'{title}의 Category는? \t')
            running_time = content.find(
                'div', class_=nm_class_RunningTime).text
            watch_date = get_date(content['data-date'])
            watch_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
            watched_YN = input(f'{title}을 끝까지 시청하셨나요? \t')

            df_record.loc[idx_rec] = [title, category,
                                      running_time, watch_date, watch_time, watched_YN]
            idx_rec += 1

        else:
            print(f"{idx}: Community Post")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search Record")

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            df_search.loc[idx_ser] = [title, search_date, search_time]

            idx_ser += 1

        else:
            print(f"{idx}: Ad")
            
            num_ad += 1


0: Watch Record
1: Ad
2: Ad
3: Search Record
4: Watch Record
5: Watch Record
6: Ad
7: Ad
8: Ad
9: Ad
10: Watch Record
11: Ad
12: Watch Record
13: Ad
14: Watch Record
15: Search Record
16: Watch Record
17: Ad
18: Watch Record
19: Watch Record
20: Ad
21: Ad
22: Watch Record
23: Watch Record
24: Ad
25: Watch Record
26: Ad
27: Watch Record
28: Ad
29: Ad
30: Watch Record
31: Ad
32: Watch Record
33: Watch Record
34: Ad
35: Search Record
36: Search Record
37: Search Record
38: Search Record
39: Search Record
40: Watch Record
41: Search Record
42: Search Record
43: Search Record
44: Search Record
45: Search Record
46: Search Record
47: Search Record
48: Search Record
49: Watch Record
50: Ad
51: Watch Record
52: Watch Record
53: Search Record
54: Ad
55: Watch Record
56: Ad
57: Search Record
58: Watch Record
59: Ad
60: Watch Record
61: Ad
62: Watch Record
63: Ad
64: Watch Record
65: Ad
66: Search Record
67: Ad
68: Watch Record
69: Watch Record
70: Ad
71: Ad
72: Watch Record
73: Ad
74: Watch Reco

In [22]:
df_record.to_csv('230205_watch_record.csv', encoding='utf-8')
df_record


,title,category,running_time,watch_date,watch_time,watched_YN
0,이직해서 절대 하면 안되는 것,노하우/스타일,12:32,2023-02-05,오전 12:09,0
1,원조의 품격 | 농심 vs. T1 게임2 하이라이트 | 02.04 | 2023 LC...,게임,11:43,2023-02-04,오후 9:33,1
2,AI가 견적낸 컴퓨터를 조립해봤는데.. 갓성비 실화입니까?!ㄷㄷ 'ChatGPT로 ...,과학기술,9:05,2023-02-04,오후 9:26,1
3,초가스 키우기 | 디알엑스 vs. 한화생명 게임1 하이라이트 | 02.04 | 20...,게임,12:33,2023-02-04,오후 8:19,1
4,신궁 | 농심 vs. T1 게임1 하이라이트 | 02.04 | 2023 LCK 스프...,게임,9:38,2023-02-04,오후 8:10,1
...,...,...,...,...,...,...
325,[오리건 트레일] 마차로 미국 대륙을 횡단하는 꿀잼 생존게임🐮 (The Oregon...,게임,5:25:26,2022-12-27,오후 1:40,0
326,AMD Ryzen 7 5700X + RTX 3060Ti - Test in 16 Games,게임?,38:46,2022-12-27,오후 1:40,1
327,RTX 3080 vs RTX 3080 Ti vs RTX 4080 Test in 7 ...,게임?,6:59,2022-12-27,오후 1:38,1
328,GTX 1060 vs RTX 3060 - How Big is the Difference?,게임?,9:49,2022-12-27,오후 1:37,1


In [24]:
df_search.to_csv('230205_search_record.csv', encoding='utf-8')
df_search

,searchword,search_date,search_time
0,뭘 많이 하긴 하는데,2023-02-04,오후 10:55
1,파카,2023-02-04,오전 2:02
2,Lck 한화,2023-02-02,오후 11:23
3,한화 승리,2023-02-02,오후 11:22
4,한화,2023-02-02,오후 11:22
...,...,...,...
128,ㅏㅇ이요,2022-12-27,오후 10:11
129,차세대,2022-12-27,오후 10:10
130,ㅊㅏ세대,2022-12-27,오후 10:10
131,Rtx3080ti vs gtx1060,2022-12-27,오후 1:36


In [32]:
print(f"입력한 시청기록 레코드는 {num_target_data}개, \n실제 시청한 영상은 {len(df_record)}개, \n광고는 {num_ad}개입니다.")

입력한 시청기록 레코드는 800개, 
실제 시청한 영상은 330개, 
광고는 330개입니다.


In [ ]:
# ERROR - 커뮤니티 글 확인 기록
# 채널은 있는데, 영상이 없음
"""
80: NO ads
지금 이 순간 대생성AI 버블 시대가 열렸습니다. 닷컴버블, 스마트폰 혁명, 블록체인 열풍을 놓치신 개발자분들 이번 기회는 꼭 잡으시길 바랍니다!
🎬영상보기: https://youtu.be/4xU0mGr4g...
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[33], line 9
      7     print(f'{idx}: NO ads')
      8     print(content.find('a', class_=nm_class_title).text)
----> 9     print(content.find('div', class_=nm_class_RunningTime).text)
     10     print(' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2]))
     12 else:

AttributeError: 'NoneType' object has no attribute 'text'
"""


### Spare codes

In [ ]:
btn_acccount_info = driver.find_element(
    By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/div[2]/div/a')

btn_sign_out = ''
# html document, iframe 태그 안에 들어있어 확인이 어려움.
# 여러 계정을 같이 분석할 경우 driver 초기화 방식으로 진행하자.


In [72]:
driver.close()


In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

contents = soup.find_all('c-wiz', class_='xDtZAf')
for idx, content in enumerate(contents):
    if content.find('div', 'SiEggd'):
        if content.find('div', class_=nm_class_RunningTime):
            print(f"{idx}: Watch Record")
            print(f"Watch Date: {get_date(content['data-date'])}")
            print(f"Title: {content.find('a', class_=nm_class_title).text}")
            print(
                f"Running Time: {content.find('div', class_=nm_class_RunningTime).text}")
            print(
                f"Watch Time: {' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])}")
        else:
            print(f"{idx}: Community Post")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search")
            print(f"Search Date: {get_date(content['data-date'])}")
            print(
                f"Search Word: {content.find('a', class_=nm_class_title).text}")
            print(
                f"Search Time: {' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])}")
        else:
            print(f"{idx}: Ad")

    print(' ')


0: Ad
 
1: Ad
 
2: Watch Record
Watch Date: 2023-02-04
Title: 초가스 키우기 | 디알엑스 vs. 한화생명 게임1 하이라이트 | 02.04 | 2023 LCK 스프링 스플릿
Running Time: 12:33
Watch Time: 오후 8:19
 
3: Ad
 
4: Ad
 
5: Watch Record
Watch Date: 2023-02-04
Title: 신궁 | 농심 vs. T1 게임1 하이라이트 | 02.04 | 2023 LCK 스프링 스플릿
Running Time: 9:38
Watch Time: 오후 8:10
 
6: Ad
 
7: Watch Record
Watch Date: 2023-02-04
Title: [스티브 잡스] A급 인재와 B급, C급 직원의 차이 (한영 자막)
Running Time: 7:27
Watch Time: 오후 8:02
 
8: Search
Search Date: 2023-02-04
Search Word: 파카
Search Time: 오전 2:02
 
9: Watch Record
Watch Date: 2023-02-04
Title: Crimson Raptor Champion Spotlight | Parody - League of Legends
Running Time: 9:45
Watch Time: 오전 1:58
 
10: Ad
 
11: Watch Record
Watch Date: 2023-02-04
Title: 젠지 vs. 리브 샌박 | 매치26 하이라이트 | 02.03 | 2023 LCK 스프링 스플릿
Running Time: 45:49
Watch Time: 오전 12:54
 
12: Watch Record
Watch Date: 2023-02-04
Title: KT vs. 광동 | 매치25 하이라이트 | 02.03 | 2023 LCK 스프링 스플릿
Running Time: 26:07
Watch Time: 오전 12:53
 
13: Ad
 
14: Ad
 
15: Watch Reco

### Troubleshooting

아이디 / 비밀번호 입력 없이 로그인을 직접 한다면?
- 보안 문제
- 로그인 한 후 driver 초기화 문제
    - 1) driver 재시작
    - 2) 